# Simple retrieval augmented generation with OpenAI

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = True

import os

os.environ['SUPERDUPER_SECRETS_VOLUME'] = '~/data/secrets/snowflake_dev'

In [2]:
from superduper import superduper, CFG
import os

db = superduper('mongomock://', initialize_cluster=False)

Building Datalayer...
Building Datalayer... DONE


In [3]:
import json
import requests
import io
from superduper import logging


def getter():
    logging.info('Downloading data...')
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    logging.info('Downloading data... (Done)')
    data = json.loads(response.content.decode('utf-8'))
    return [{'x': r} for r in data]

In [4]:
if APPLY:
    data = getter()

In [5]:
from superduper import Table

table = Table('<var:table_name>', fields={'x': 'str'})

Create plugin:

In [6]:
from superduper import Plugin

plugin = Plugin(path='./rag_plugin.py')

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [7]:
from superduper import Listener
from rag_plugin import Chunker

upstream_listener = Listener(
    model=Chunker(identifier='chunker'),
    select=db['<var:table_name>'],
    key='x',
    identifier='chunker',
    flatten=True,
    upstream=[plugin],
)

<!-- TABS -->
## Build text embedding model

OpenAI:

In [8]:
import os

from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(
    identifier='text-embedding',
    model='<var:embedding_model>',
    datatype='vector[float32:1536]',
)

## Create vector-index

In [9]:
from superduper import VectorIndex, Listener

vector_index_name = 'vectorindex'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs],
        model=openai_embedding,
        identifier='embeddinglistener',
        upstream=[upstream_listener],
    )
)

<!-- TABS -->
## Build LLM

In [10]:
from superduper_openai import OpenAIChatCompletion


llm_openai = OpenAIChatCompletion(
    identifier='llm-model',
    model='<var:llm_model>',
    datatype='str',
)

## Answer question with LLM

In [11]:
from rag_plugin import RAGModel


prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "These snippets are samples from our internal data-repositories, and should be used exclusively and as a matter"
    " of priority to answer the question. Please answer in 20 words or less.\n"
    "{context}\n"
    "Here is the question: {query}"
)


rag = RAGModel(
    'simple_rag',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm_openai,
)

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [12]:
from superduper import Streamlit, Plugin
from rag_plugin import demo_func

demo = Streamlit('simple-rag-demo', demo_func=demo_func)

In [16]:
from superduper import Application

TABLE = 'other'

app = Application(
    'simple-rag-app',
    upstream=[table, plugin],
    components=[
        upstream_listener,
        vector_index,
        rag,
        demo,
    ],
    variables={
        'table_name': TABLE,
        'id_field': '_id',
        'embedding_model': 'text-embedding-ada-002',
        'llm_model': 'gpt-3.5-turbo',
    }
)

In [17]:
app.show()

Application:simple-rag-app:6c4f73b9d964d6faeb
├── upstream
│   ├── [0] Table:other:123ae67eab75faf8b7
│   │   ├── fields: {'x': 'str'}
│   │   └── primary_id: id
│   └── [1] Plugin:plugin-rag_plugin_py:8624439cb46a595e50
│       ├── path: /Users/dodo/.superduper/plugins/ddfb21fd92ba14e2ad/rag_plugin.py
│       └── cache_path: ~/.superduper/plugins
├── components
│   ├── [0] Listener:chunker:1862a8fc1629f85fe6
│   │   ├── upstream
│   │   │   └── [0] Plugin:plugin-rag_plugin_py:49e0c72d2b166dbd05
│   │   │       ├── path: /Users/dodo/.superduper/plugins/dcdf7aa2a2895ea7e8/rag_plugin.py
│   │   │       └── cache_path: ~/.superduper/plugins
│   │   ├── cdc_table: docs
│   │   ├── key: x
│   │   ├── model: Chunker:chunker:b12cd565989b72fc4c
│   │   │   ├── datatype: str
│   │   │   └── chunk_size: 200
│   │   ├── select: docs
│   │   └── flatten: True
│   ├── [1] VectorIndex:vectorindex:851e610e9d2ace0b54
│   │   ├── cdc_table: _outputs__embeddinglistener__a67199bb7c0c1ac162
│   │   ├── indexing_listener: Listener:embeddinglistener:a67199bb7c0c1ac162
│   │   │   ├── upstream
│   │   │   │   └── [0] Listener:chunker:1862a8fc1629f85fe6
│   │   │   │       ├── upstream
│   │   │   │       │   └── [0] Plugin:plugin-rag_plugin_py:5091ab959553035f51
│   │   │   │       │       ├── path: /Users/dodo/.superduper/plugins/01e925c9d5ced65a17/rag_plugin.py
│   │   │   │       │       └── cache_path: ~/.superduper/plugins
│   │   │   │       ├── cdc_table: docs
│   │   │   │       ├── key: x
│   │   │   │       ├── model: Chunker:chunker:b12cd565989b72fc4c
│   │   │   │       │   ├── datatype: str
│   │   │   │       │   └── chunk_size: 200
│   │   │   │       ├── select: docs
│   │   │   │       └── flatten: True
│   │   │   ├── cdc_table: _outputs__chunker__1862a8fc1629f85fe6
│   │   │   ├── key: _outputs__chunker__1862a8fc1629f85fe6
│   │   │   ├── model: OpenAIEmbedding:text-embedding:2b907268839faa545d
│   │   │   │   ├── datatype: vector
│   │   │   │   ├── model: text-embedding-ada-002
│   │   │   │   ├── max_batch_size: 8
│   │   │   │   └── batch_size: 100
│   │   │   └── select: _outputs__chunker__1862a8fc1629f85fe6
│   │   └── measure: cosine
│   ├── [2] RAGModel:simple_rag:233661fe822e51dd90
│   │   ├── datatype: str
│   │   ├── prompt_template: Use the following context snippets, these snippets are not ordered!, Answer the 
│   │   │   question based on this context.
│   │   │   These snippets are samples from our internal data-repositories, and should be used exclusively and as a
│   │   │   matter of priority to answer the question. Please answer in 20 words or less.
│   │   │   {context}
│   │   │   Here is the question: {query}
│   │   ├── select: _outputs__chunker__1862a8fc1629f85fe6.select().like({'_outputs__chunker__1862a8fc1629f85fe6': 
│   │   │   '<var:query>'}, "vectorindex", n=5)
│   │   ├── key: _outputs__chunker__1862a8fc1629f85fe6
│   │   └── llm: OpenAIChatCompletion:llm-model:ffce2ddfa3a6a5288f
│   │       ├── datatype: str
│   │       ├── model: gpt-3.5-turbo
│   │       ├── max_batch_size: 8
│   │       └── batch_size: 1
│   └── [3] Streamlit:simple-rag-demo:b33aaf6fa76d173336
│       └── demo_func: <function demo_func at 0x11ffdd870>
└── variables: {'table_name': 'other', 'id_field': '_id', 'embedding_model': 'text-embedding-ada-002', 'llm_model':
    'gpt-3.5-turbo'}

In [15]:
if APPLY:
    db.apply(app)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Component to apply                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Application:simple-rag-app:6c4f73b9d964d6faeb
├── upstream
│   ├── [0] Table:docs:3b21dc32e8beebd92b
│   │   ├── fields: {'x': 'str'}
│   │   └── primary_id: id
│   └── [1] Plugin:plugin-rag_plugin_py:8624439cb46a595e50
│       ├── path: /Users/dodo/.superduper/plugins/ddfb21fd92ba14e2ad/rag_plugin.py
│       └── cache_path: ~/.superduper/plugins
├── components
│   ├── [0] Listener:chunker:1862a8fc1629f85fe6
│   │   ├── upstream
│   │   │   └── [0] Plugin:plugin-rag_plugin_py:dcdf7aa2a2895ea7e8
│   │   │       ├── path: /Users/dodo/.superduper/plugins/64a9436d940ae35d51/rag_plugin.py
│   │   │       └── cache_path: ~/.superduper/plugins
│   │   ├── cdc_table: docs
│   │   ├── key: x
│   │   ├── model: Chunker:chunker:b12cd565989b72fc4c
│   │   │   ├── datatype: str
│   │   │   └── chunk_size: 200
│   │   ├── select: docs
│   │   └── flatten: True
│   ├── [1] VectorIndex:vectorindex:851e610e9d2ace0b54
│   │   ├── cdc_table: _outputs__embeddinglistener__a67199bb7c0c1ac162
│   │   ├── indexing_listener: Listener:embeddinglistener:a67199bb7c0c1ac162
│   │   │   ├── upstream
│   │   │   │   └── [0] Listener:chunker:1862a8fc1629f85fe6
│   │   │   │       ├── upstream
│   │   │   │       │   └── [0] Plugin:plugin-rag_plugin_py:ba47e61be1866e7cf2
│   │   │   │       │       ├── path: /Users/dodo/.superduper/plugins/83e1c986d78eea7005/rag_plugin.py
│   │   │   │       │       └── cache_path: ~/.superduper/plugins
│   │   │   │       ├── cdc_table: docs
│   │   │   │       ├── key: x
│   │   │   │       ├── model: Chunker:chunker:b12cd565989b72fc4c
│   │   │   │       │   ├── datatype: str
│   │   │   │       │   └── chunk_size: 200
│   │   │   │       ├── select: docs
│   │   │   │       └── flatten: True
│   │   │   ├── cdc_table: _outputs__chunker__1862a8fc1629f85fe6
│   │   │   ├── key: _outputs__chunker__1862a8fc1629f85fe6
│   │   │   ├── model: OpenAIEmbedding:text-embedding:2b907268839faa545d
│   │   │   │   ├── datatype: vector
│   │   │   │   ├── model: text-embedding-ada-002
│   │   │   │   ├── max_batch_size: 8
│   │   │   │   └── batch_size: 100
│   │   │   └── select: _outputs__chunker__1862a8fc1629f85fe6
│   │   └── measure: cosine
│   ├── [2] RAGModel:simple_rag:233661fe822e51dd90
│   │   ├── datatype: str
│   │   ├── prompt_template: Use the following context snippets, these snippets are not ordered!, Answer the 
│   │   │   question based on this context.
│   │   │   These snippets are samples from our internal data-repositories, and should be used exclusively and as a
│   │   │   matter of priority to answer the question. Please answer in 20 words or less.
│   │   │   {context}
│   │   │   Here is the question: {query}
│   │   ├── select: _outputs__chunker__1862a8fc1629f85fe6.select().like({'_outputs__chunker__1862a8fc1629f85fe6': 
│   │   │   '<var:query>'}, "vectorindex", n=5)
│   │   ├── key: _outputs__chunker__1862a8fc1629f85fe6
│   │   └── llm: OpenAIChatCompletion:llm-model:ffce2ddfa3a6a5288f
│   │       ├── datatype: str
│   │       ├── model: gpt-3.5-turbo
│   │       ├── max_batch_size: 8
│   │       └── batch_size: 1
│   └── [3] Streamlit:simple-rag-demo:b33aaf6fa76d173336
│       └── demo_func: <function demo_func at 0x11ffdd870>
└── variables: {'table_name': 'docs', 'id_field': '_id', 'embedding_model': 'text-embedding-ada-002', 'llm_model': 
    'gpt-3.5-turbo'}

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Deployment plan                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────── Deployment plan – events overview ───────────────────────────────────────╮
│ ┏━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ #  ┃ Event type ┃ Details                                                                                   ┃ │
│ ┡━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ 0  │ TABLE      │ docs                                                                                      │ │
│ │ 1  │ TABLE      │ _outputs__chunker__1862a8fc1629f85fe6[_outputs__chunker__1862a8fc1629f85fe6=str;_source=… │ │
│ │ 2  │ TABLE      │ _outputs__embeddinglistener__a67199bb7c0c1ac162[_outputs__embeddinglistener__a67199bb7c0… │ │
│ │ 3  │ CREATE     │ Table:docs:3b21dc32e8beebd92b: Create                                                     │ │
│ │ 4  │ CREATE     │ Plugin:plugin-rag_plugin_py:8624439cb46a595e50: Create                                    │ │
│ │ 5  │ CREATE     │ Plugin:plugin-rag_plugin_py:dcdf7aa2a2895ea7e8: Create                                    │ │
│ │ 6  │ CREATE     │ Chunker:chunker:b12cd565989b72fc4c: Create                                                │ │
│ │ 7  │ CREATE     │ Table:_outputs__chunker__1862a8fc1629f85fe6:55813a15e4d7f63d2b: Create                    │ │
│ │ 8  │ CREATE     │ Listener:chunker:1862a8fc1629f85fe6: Create                                               │ │
│ │ 9  │ CREATE     │ OpenAIEmbedding:text-embedding:2b907268839faa545d: Create                                 │ │
│ │ 10 │ CREATE     │ Table:_outputs__embeddinglistener__a67199bb7c0c1ac162:392e97d67fb77a15dd: Create          │ │
│ │ 11 │ CREATE     │ Listener:embeddinglistener:a67199bb7c0c1ac162: Create                                     │ │
│ │ 12 │ CREATE     │ VectorIndex:vectorindex:851e610e9d2ace0b54: Create                                        │ │
│ │ 13 │ CREATE     │ OpenAIChatCompletion:llm-model:ffce2ddfa3a6a5288f: Create                                 │ │
│ │ 14 │ CREATE     │ RAGModel:simple_rag:233661fe822e51dd90: Create                                            │ │
│ │ 15 │ CREATE     │ Streamlit:simple-rag-demo:b33aaf6fa76d173336: Create                                      │ │
│ │ 16 │ CREATE     │ Application:simple-rag-app:6c4f73b9d964d6faeb: Create                                     │ │
│ │ 17 │ PUT        │ Chunker:chunker:b12cd565989b72fc4c/compute                                                │ │
│ │ 18 │ PUT        │ Listener:chunker:1862a8fc1629f85fe6/scheduler                                             │ │
│ │ 19 │ PUT        │ Listener:chunker:1862a8fc1629f85fe6/cdc                                                   │ │
│ │ 20 │ PUT        │ OpenAIEmbedding:text-embedding:2b907268839faa545d/compute                                 │ │
│ │ 21 │ PUT        │ Listener:embeddinglistener:a67199bb7c0c1ac162/scheduler                                   │ │
│ │ 22 │ PUT        │ Listener:embeddinglistener:a67199bb7c0c1ac162/cdc                                         │ │
│ │ 23 │ PUT        │ VectorIndex:vectorindex:851e610e9d2ace0b54/vector_search                                  │ │
│ │ 24 │ PUT        │ OpenAIChatCompletion:llm-model:ffce2ddfa3a6a5288f/compute                                 │ │
│ │ 25 │ PUT        │ RAGModel:simple_rag:233661fe822e51dd90/compute                                            │ │
│ │ 26 │ JOB        │ Table:docs:3b21dc32e8beebd92b.set_status: set_status                                      │ │
│ │ 27 │ JOB        │ Plugin:plugin-rag_plugin_py:8624439cb46a595e50.set_status: set_status                     │ │
│ │ 28 │ JOB        │ Plugin:plugin-rag_plugin_py:dcdf7aa2a2895ea7e8.set_status: set_status                     │ │
│ │ 29 │ JOB        │ Chunker:chunker:b12cd565989b72fc4c.set_status: set_status                                 │ │
│ │ 30 │ JOB        │ Table:_outputs__chunker__1862a8fc1

Please approve this deployment plan. [Y/n]:

In [ ]:
if APPLY:
    db[TABLE].insert(data)

In [ ]:
app.export('.', format='json')